In [ ]:
! pip install -q tensorflow_ranking
! pip install -q requests

import json
import os
import requests


import tensorflow as tf
import tensorflow_ranking as tfr

from google.protobuf import text_format
from tensorflow_ranking.python.keras import saved_model as saved_model_lib
from tensorflow_serving.apis import input_pb2

# Create TFR Model

In [ ]:
# Define constants.
EXPORT_DIR = '/tmp/tfr_dnn_ranking_model'
MASK_FEATURE_NAME = 'mask'
EXAMPLE_FEATURE_SPEC = {
        name:
        tf.io.FixedLenFeature(shape=(1,), dtype=tf.float32, default_value=0.0)
        for name in ["feature_1", "feature_2"]
    }

In [ ]:
input_creator = tfr.keras.model.FeatureSpecInputCreator(
    context_feature_spec={},
    example_feature_spec=EXAMPLE_FEATURE_SPEC,
)
dnn_scorer = tfr.keras.model.DNNScorer(
    hidden_layer_dims=[10, 10],
    output_units=1
)
dnn_model = tfr.keras.model.ModelBuilder(
    input_creator=input_creator,
    preprocessor=tfr.keras.model.PreprocessorWithSpec(),
    scorer=dnn_scorer,
    mask_feature_name=MASK_FEATURE_NAME,
    name='test_model'
).build()

In [ ]:
def _export(model, filepath, serving_default="regress"):
  signatures = saved_model_lib.Signatures(
      model=model,
      context_feature_spec={},
      example_feature_spec=EXAMPLE_FEATURE_SPEC,
      mask_feature_name=MASK_FEATURE_NAME)(serving_default)
  model.save(filepath=filepath, signatures=signatures, save_format="tf")

_export(dnn_model, EXPORT_DIR + '/1/')

# Check that model can be loaded.
loaded_model = tf.saved_model.load(export_dir=EXPORT_DIR + '/1/')

INFO:tensorflow:Assets written to: /tmp/tfr_dnn_ranking_model/1/assets


INFO:tensorflow:Assets written to: /tmp/tfr_dnn_ranking_model/1/assets


# Start a TensorFlow Server

In [ ]:
!wget 'http://storage.googleapis.com/tensorflow-serving-apt/pool/tensorflow-model-server-universal-2.8.0/t/tensorflow-model-server-universal/tensorflow-model-server-universal_2.8.0_all.deb'
!dpkg -i tensorflow-model-server-universal_2.8.0_all.deb

--2022-06-28 21:38:54--  http://storage.googleapis.com/tensorflow-serving-apt/pool/tensorflow-model-server-universal-2.8.0/t/tensorflow-model-server-universal/tensorflow-model-server-universal_2.8.0_all.deb
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.123.128, 142.250.98.128, 142.250.97.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.123.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 335421916 (320M) [application/x-debian-package]
Saving to: ‘tensorflow-model-server-universal_2.8.0_all.deb.1’

tensorflow-model-se 100%[===================>] 319.88M   308MB/s    in 1.0s    

2022-06-28 21:38:55 (308 MB/s) - ‘tensorflow-model-server-universal_2.8.0_all.deb.1’ saved [335421916/335421916]

(Reading database ... 155640 files and directories currently installed.)
Preparing to unpack tensorflow-model-server-universal_2.8.0_all.deb ...
Unpacking tensorflow-model-server-universal (2.8.0) over (2.8.0) ...
Setting

In [ ]:
import os
os.environ["MODEL_DIR"] = EXPORT_DIR

! nohup tensorflow_model_server \
  --rest_api_port=8502 \
  --model_name="tfr_model" \
  --model_base_path="/tmp/tfr_dnn_ranking_model" >server.log 2>&1 &

In [ ]:
! tail server.log

[warn] getaddrinfo: address family for nodename not supported
[evhttp_server.cc : 245] NET_LOG: Entering the event loop ...
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

## Send Regress Requests to the Server

In [ ]:
headers = {"content-type": "application/json"}

data = json.dumps({
    "signature_name": tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY, 
    "examples": [
                 {"feature_1": [1.], "feature_2": [2.]},
                 {"feature_1": [3.], "feature_2": [4.]},  
                 ],
    })
json_response = requests.post(
    'http://localhost:8502/v1/models/tfr_model:regress', 
    data=data, headers=headers)
predictions = json.loads(json_response.text)
print(predictions)

{'results': [2.49908638, 4.51557446]}
